In [1]:
from datetime import datetime
import math
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf

In [2]:
def loadfile(filename):
    pickle_in = open("C:/Users/Akarsh/Downloads/DP_scripts/store_emb/" + filename + ".pickle","rb")
    return pickle.load(pickle_in)

def save2file(filename, data):
    pickle_out = open("C:/Users/Akarsh/Downloads/DP_scripts/store_emb/" + filename + ".pickle", "wb")
    pickle.dump(data, pickle_out, protocol=pickle.HIGHEST_PROTOCOL)
    pickle_out.close()

main_str = str(437)

filename = "ip2vec_emb_" + main_str
ip2vec_emb = loadfile(filename)
print(ip2vec_emb.shape)

# public wgan
MODE = "dp_wgan_gp" #"wgan_gp"
# filename = MODE + epch_str 
# filename = MODE + '_final_' + main_str
filename =  "dp_wgan_gp_epoch_9"

gan_emb = loadfile(filename)
print(gan_emb.shape)
display(ip2vec_emb.head())

(58863, 21)
(260391, 160)


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,values
0,0.374719,0.242699,0.804956,0.233550,0.753552,0.161305,0.899202,0.471473,0.639400,0.082626,...,0.585408,0.393872,0.928697,0.461868,0.389630,0.020627,0.514851,0.879726,0.615675,TCP
1,0.690727,0.289209,0.624222,0.787160,0.259145,1.000000,0.844552,0.869673,0.398222,0.751319,...,1.000000,0.432327,0.184115,0.275347,0.720820,0.338340,0.552738,0.503326,0.304589,0.0_d
2,0.700514,0.424836,0.185812,0.616382,0.469560,0.492044,0.000000,0.564201,0.792898,0.874603,...,0.329597,0.540815,0.640496,0.986693,0.467639,0.614137,0.588136,0.802519,0.292566,1_k
3,0.000000,0.229812,0.721760,0.847710,0.425830,0.227527,0.530943,0.329691,0.724129,0.800538,...,0.757808,0.089902,0.220831,0.921574,0.741282,0.982664,0.848599,0.535690,0.367750,443_p
4,0.197396,0.000000,0.483233,0.585403,0.172215,0.472147,0.425286,0.326074,0.365241,0.838361,...,0.956589,0.395134,0.096285,0.809280,0.605448,0.637512,1.000000,0.906251,0.830612,80_p


In [3]:
ipToEmbed = dict();   portToEmbed = dict(); byteToEmbed = dict(); 
packToEmbed = dict(); durToEmbed = dict();  embSizeIP = -1;

for row in ip2vec_emb.to_numpy():
    key = row[-1]
    embSizeIP = len(row) - 1 #21 cols
    if key.count('.') > 2:
        ipToEmbed[key] = np.asarray(row).reshape(1,-1)
    elif "_p" in key:
        key = key.split("_")[0]
        portToEmbed[key] = np.asarray(row).reshape(1,-1)
    elif "_k" in key:
        key = key.split("_")[0]
        packToEmbed[key] = np.asarray(row).reshape(1,-1)
    elif "_b" in key:
        key = key.split("_")[0]
        byteToEmbed[key] = np.asarray(row).reshape(1,-1)
    elif "_d" in key:
        key = key.split("_")[0]
        durToEmbed[key] = np.asarray(row).reshape(1,-1)

In [4]:
def compute_norm(emb_dict, feature_ip, feature_name):
    feature_emb = []; feature_label = []; comp_id = -1;
    for k, v in emb_dict.items():
        feature_label.append(k)
        feature_emb.append(v[0][:20])
        if feature_ip == 'yes':
            if k == "192.168.220.255":
                comp_id = len(feature_emb) - 1 # count broad/multi cast IP
        else:
            continue
    emb_arr = np.reshape(np.array(feature_emb, dtype=np.float32) , (len(emb_dict), embSizeIP)) 
    norm_feature = np.ndarray(shape=(1, len(emb_dict)), dtype=np.float32)
    for i in range(0, len(emb_arr)):
        line = emb_arr[i]
        res = 0
        for value in line:
            res += (value*value)
        res = math.sqrt(res)
        norm_feature[0][i] = res
    print(feature_name, ':', emb_arr.shape, norm_feature.shape, feature_label[:3])
    return emb_arr, norm_feature, feature_label

embedding_ip, ip_norm, feature_label_ip = compute_norm(ipToEmbed, feature_ip='yes', feature_name='ip addresses')
embedding_prt, prt_norm, feature_label_prt = compute_norm(portToEmbed, feature_ip='no', feature_name='port numbers')
embedding_byt, byt_norm, feature_label_byt = compute_norm(byteToEmbed, feature_ip='no', feature_name='bytes')
embedding_pkt, pkt_norm, feature_label_pkt = compute_norm(packToEmbed, feature_ip='no', feature_name='packets')
embedding_dur, dur_norm, feature_label_dur = compute_norm(durToEmbed, feature_ip='no', feature_name='duration')
print('vocab size:', (embedding_ip.shape[0] + embedding_prt.shape[0] + embedding_byt.shape[0] + embedding_pkt.shape[0] + embedding_dur.shape[0]))

ip addresses : (149, 20) (1, 149) ['192.168.220.15', '192.168.220.16', '192.168.210.5']
port numbers : (33231, 20) (1, 33231) ['443', '80', '53']
bytes : (12650, 20) (1, 12650) ['66', '55', '54']
packets : (577, 20) (1, 577) ['1', '2', '3']
duration : (3550, 20) (1, 3550) ['0.0', '0.003', '0.002']
vocab size: 50157


In [5]:
# IP 
ip_row = tf.placeholder(tf.float32,shape=[1,embSizeIP])
ip_mat = tf.placeholder(tf.float32,shape=[len(embedding_ip),embSizeIP])
ip_nor = tf.placeholder(tf.float32,shape=[1,len(embedding_ip)])
ip_sim = tf.matmul(ip_row,ip_mat,transpose_b=True)
ip_res = tf.math.divide(ip_sim,ip_nor)

# Ports
pt_row = tf.placeholder(tf.float32,shape=[1,embSizeIP])
pt_mat = tf.placeholder(tf.float32,shape=[len(embedding_prt),embSizeIP])
pt_nor = tf.placeholder(tf.float32,shape=[1,len(embedding_prt)])
pt_sim = tf.matmul(pt_row,pt_mat,transpose_b=True)
pt_res = tf.math.divide(pt_sim,pt_nor)

# Byte
by_row = tf.placeholder(tf.float32,shape=[1,embSizeIP])
by_mat = tf.placeholder(tf.float32,shape=[len(embedding_byt),embSizeIP]) 
by_nor = tf.placeholder(tf.float32,shape=[1,len(embedding_byt)]) 
by_sim = tf.matmul(by_row,by_mat,transpose_b=True) 
by_res = tf.math.divide(by_sim,by_nor)

# Packets
pk_row = tf.placeholder(tf.float32,shape=[1,embSizeIP]) 
pk_mat = tf.placeholder(tf.float32,shape=[len(embedding_pkt),embSizeIP])
pk_nor = tf.placeholder(tf.float32,shape=[1,len(embedding_pkt)])
pk_sim = tf.matmul(pk_row,pk_mat,transpose_b=True)
pk_res = tf.math.divide(pk_sim,  pk_nor)

# Duration
du_row = tf.placeholder(tf.float32,shape=[1,embSizeIP])
du_mat = tf.placeholder(tf.float32,shape=[len(embedding_dur),embSizeIP])
du_nor = tf.placeholder(tf.float32,shape=[1,len(embedding_dur)])
du_sim = tf.matmul(du_row, du_mat,transpose_b=True) 
du_res = tf.math.divide(du_sim, du_nor)

sess = tf.Session()
init = tf.global_variables_initializer()

def findIP(vec):
    res = sess.run([ip_res], feed_dict = {ip_row: vec, ip_mat: embedding_ip, ip_nor: ip_norm})
    return feature_label_ip[ res[0].argmax() ]

def findPt(vec):
    res = sess.run([pt_res], feed_dict = {pt_row: vec, pt_mat: embedding_prt, pt_nor: prt_norm})
    return feature_label_prt[ res[0].argmax() ]

def findBy(vec):
    res = sess.run([by_res], feed_dict = {by_row: vec, by_mat: embedding_byt, by_nor: byt_norm})
    return feature_label_byt[ res[0].argmax() ]

def findPk(vec):
    res = sess.run([pk_res], feed_dict = {pk_row: vec, pk_mat: embedding_pkt, pk_nor: pkt_norm})
    return feature_label_pkt[ res[0].argmax() ]

def findDu(vec):
    res = sess.run([du_res], feed_dict = {du_row: vec, du_mat: embedding_dur, du_nor: dur_norm})
    return feature_label_dur[ res[0].argmax() ]

In [6]:
counter_read = 0; decoded_gan = []
for row in gan_emb:
    counter_read += 1
    if counter_read % 50000 == 0:
        print("Read lines: ", str(counter_read))
    rl = []
    d_id = np.array(row[0:7],dtype=np.float32).argmax()
    rl.append(d_id)

    time = (float( row[7] )) * 86400
    if time < 0:
        time += 86400 
    if time > 86399: 
        time = time % 86400
    h = int( time / 3600 ) 
    m = int( (time-(h*3600)) / 60 ) 
    s = round( (time-(h*3600)-(m*60)) ) 
    rl.append(str(h)+":"+str(m)+":"+str(s))
    
    # Duration
    du = row[8:8+embSizeIP]
    rl.append(findDu( np.array(du).reshape(1,embSizeIP)) )

    # Transport Protocol
    p_id = np.array(row[8+embSizeIP:8+embSizeIP+3],dtype=np.float32).argmax() 
    if p_id == 0: 
        rl.append("ICMP")
    if p_id == 1: 
        rl.append("UDP")
    if p_id == 2: 
        rl.append("TCP")
    
    sip = row[11+embSizeIP:11+2*embSizeIP]
    rl.append(findIP( np.array(sip).reshape(1,embSizeIP)) ) 

    sp = row[11+2*embSizeIP:11+3*embSizeIP]
    rl.append(findPt( np.array(sp).reshape(1,embSizeIP)) ) 

    dip = row[11+3*embSizeIP:11+4*embSizeIP]
    rl.append(findIP( np.array(dip).reshape(1,embSizeIP)) ) 

    dp = row[11+4*embSizeIP:11+5*embSizeIP]
    rl.append(findPt( np.array(dp).reshape(1,embSizeIP))) 

    pk = row[11+5*embSizeIP:11+6*embSizeIP]
    rl.append(findPk( np.array(pk).reshape(1,embSizeIP)) ) 

    by = row[11+6*embSizeIP:11+7*embSizeIP]
    rl.append(findBy( np.array(by).reshape(1,embSizeIP)) ) 

    # Read the flags 
    synFlag     = ( float( row[11+7*embSizeIP+0] ) )
    ackFlag     = ( float( row[11+7*embSizeIP+1] ) )
    finFlag     = ( float( row[11+7*embSizeIP+2] ) )
    urgentFlag  = ( float( row[11+7*embSizeIP+3] ) )
    pushFlag    = ( float( row[11+7*embSizeIP+4] ) )
    resetFlag   = ( float( row[11+7*embSizeIP+5] ) )
    flags = "" 
    if urgentFlag > 0.5:
        flags += "U"
    else:
        flags += "."
    if ackFlag > 0.5:
        flags += "A" 
    else: 
        flags += "." 
    if pushFlag > 0.5:  
        flags += "P" 
    else: 
        flags += "." 
    if resetFlag > 0.5:  
        flags += "R" 
    else: 
        flags += "." 
    if synFlag > 0.5: 
        flags += "S" 
    else: 
        flags += "."
    if finFlag > 0.5:  
        flags += "F" 
    else: 
        flags += "."
    rl.append(flags)   

    # clazz 
    normal = 0.0; attacker = 0.0; victim = 0.0
    c_id = np.array(row[11+7*embSizeIP+6:11+7*embSizeIP+6+3], dtype=np.float32).argmax() 
    if c_id == 0:
        rl.append("normal")
    if c_id == 1: 
        rl.append("attacker")
    if c_id == 2: 
        rl.append("victim")
    decoded_gan.append(rl)

Read lines:  50000
Read lines:  100000
Read lines:  150000
Read lines:  200000
Read lines:  250000


In [7]:
col_names = ['Day', 'Time', 'Duration', 'Proto', 'Src IP Addr', 'Src Pt', 'Dst IP Addr', 'Dst Pt', 'Packets', 'Bytes', 'Flags', 'class']
final_output = pd.DataFrame(decoded_gan, columns = col_names)
display(final_output.head())
print(final_output.shape)

filename = "final_output_" + main_str
save2file(filename, final_output)

,Day,Time,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class
0,4,10:35:34,0.0,TCP,192.168.220.255,80,192.168.220.14,48311,1,66,.A....,normal
1,4,10:49:56,0.0,TCP,192.168.200.1,443,192.168.200.4,53991,1,66,.A....,normal
2,4,15:24:33,0.042,TCP,192.168.220.6,41724,192.168.220.2,80,12,9135,.AP.S.,normal
3,0,16:38:58,0.139,TCP,192.168.220.6,47063,192.168.200.1,443,20,5652,.AP.S.,normal
4,2,15:15:22,0.0,TCP,192.168.100.2,443,192.168.220.4,45052,1,66,.A....,normal


(260391, 12)
